# G211 gridpoints inside CONUS shapes
<li> G211 module pickle mask
    <li> Natural Earth low res
<li> share/shapeFiles/CONUS

In [ ]:
import cartopy.crs as ccrs
import geopandas
import matplotlib.pyplot as plt
import numpy as np
import xarray
from shapely.geometry import Polygon

from ahijevyc import G211
from statisticplot import make_map

<div>
    <img src="https://www.nco.ncep.noaa.gov/pmb/docs/on388/grids/grid211.gif", width="50%"/>
</div>

In [ ]:
grid = G211
grid.grid.crs

# Naturalearth lowres USA - overlay lat/lon box around CONUS

In [ ]:
poly = geopandas.GeoDataFrame.from_file(geopandas.datasets.get_path("naturalearth_lowres"))
usa = poly[poly.iso_a3 == "USA"]
usa.plot()

In [ ]:
# lat/lon box around CONUS (no AK or HI)
lat_point_list = [51, 51, 20, 20, 51]
lon_point_list = [-130, -60, -60, -130, -130]

polygon_geom = Polygon(zip(lon_point_list, lat_point_list))
polygon = geopandas.GeoDataFrame(crs=usa.crs, geometry=[polygon_geom])
conus = usa.overlay(polygon, how="intersection")
conus.plot()

In [ ]:
pointInPolys = grid.grid.to_crs(conus.crs).sjoin(conus)

fig, ax = make_map(bbox=[-125, -67, 20, 48])
conus.geometry.plot(ax=ax, color="#DDDDDD", transform=ccrs.PlateCarree())
pointInPolys.plot(ax=ax, alpha=0.6, transform=ccrs.PlateCarree(), label="conus poly mask")

g211_obj = grid.grid[grid.mask.values.flatten()].plot(
    ax=ax, alpha=0.5, transform=ccrs.PlateCarree(), label="g211 mask"
)
leg = plt.legend()

In [ ]:
import folium

pointInPolys = grid.grid.to_crs(conus.crs).sjoin(conus)
m = pointInPolys.explore(marker_kwds=dict(radius=4), name="conus poly mask")
grid.grid[grid.mask.values.flatten()].explore(
    m=m, marker_kwds=dict(color="orange"), name="g211 mask"
)
folium.LayerControl().add_to(m)
m

# share/shapeFiles/CONUS minus HI, AK, territories

In [ ]:
poly = geopandas.GeoDataFrame.from_file("/glade/work/ahijevyc/share/shapeFiles/CONUS")
# all STUSPS except these
conus = poly[~poly.STUSPS.isin(["HI", "PR", "VI", "AK", "GU", "MP", "AS"])]
conus.plot()

In [ ]:
pointInPolys = grid.grid.sjoin(conus.to_crs(grid.grid.crs))
pointInPolys.head()

In [ ]:
m = pointInPolys.explore(marker_kwds=dict(radius=4), name="conus poly mask")
grid.grid[grid.mask.values.flatten()].explore(
    m=m, marker_kwds=dict(color="orange"), name="g211 mask"
)
folium.LayerControl().add_to(m)
m

In [ ]:
# dx times dy = area
a = np.diff(grid.xv, append=np.nan) * np.diff(grid.yv, axis=0, append=np.nan)
a /= 1000**2
a

In [ ]:
np.nanmin(a), np.nanmax(a)

In [ ]:
np.diff(grid.xv).min(), np.diff(grid.xv).max()

In [ ]:
np.diff(grid.yv, axis=0).min(), np.diff(grid.yv, axis=0).max()

In [ ]:
# Load lat/lon from netCDF
ifile = "/glade/campaign/mmm/parc/schwartz/HWT2024/mpas/2024052200/post/mem_1/interp_mpas_3km_2024052200_mem1_f001.nc"
ds = xarray.open_dataset(ifile)
df = ds["refl10cm_max"].to_dataframe().reset_index()

In [ ]:
mgrid = geopandas.GeoDataFrame(
    df.refl10cm_max,
    geometry=geopandas.points_from_xy(
        ds.longitude.values.ravel(), ds.latitude.values.ravel(), crs="EPSG:4326"
    ),
)
mgrid.crs

In [ ]:
grid.grid.crs

In [ ]:
gdf = mgrid.sjoin(conus.to_crs(mgrid.crs))
len(gdf)

In [ ]:
gdf.plot()

In [ ]:
conus.dissolve().plot()  # .to_crs(mgrid.crs).contains(mgrid)

In [ ]:
mgrid.head()

In [ ]:
from tqdm import tqdm

conus1 = conus.dissolve()
conus_mask = np.array([g.within(conus1.geometry.values[0]) for g in tqdm(mgrid.geometry)])
conus_mask.sum()

In [ ]:
nlat, nlon = ds.lat.size, ds.lon.size
conus_mask = xarray.DataArray(
    conus_mask.reshape(nlat, nlon),
    dims=["lat", "lon"],
    coords={"lat": range(nlat), "lon": range(nlon)},
)
conus_mask.plot()

In [ ]:
t = conus_mask.__xarray_dataarray_variable__
t.name = None
conus_mask.to_netcdf("HWT_2024.conus.nc")